## ANALYZE
the data

This is a work in progress  

Vectorize the data in beer.review  
Diminish the importance of common words  
Compare ML algorithms to use the review data to predict beer.style  

In [27]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

pd.set_option('max_colwidth', 300)

df = pd.read_csv('beer.csv', header=0)
df_copy = df  #save a copy of dataframe for reference. 
print('length',len(df))

#prep the data
df = df.drop(['name','brewery'], axis=1)
# drop all reviews with < 15 characters
df = df[df['review'].map(len) > 14]
print('length',len(df))

X = df['review'].values
y = df['style'].values
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=22)

print('  total reviews:', format(len(df), ',d'))
df.head(3)

length 80818
length 49291
  total reviews: 49,291


,style,rating,review
0,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sh..."
1,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much old ale characteristic. too light for that. its still tasty. the cardamom does add...
2,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. nothing almond came out of it that was obvious. kind of a fancied up oktoberfest ..."


In [28]:
print('X_train:',len(X_train), 'y_train:',len(y_train))
print('X_test:', len(X_test), 'y_test:', len(y_test))

X_train: 34503 y_train: 34503
X_test: 14788 y_test: 14788


In [29]:
# Change review to a string of words.  remove non-letters, make lower case, split into words.  
# Remove stopwords (common words.)  Join back together into a long sentence.  

def review_to_words(review):
    letters_only = re.sub('[^a-zA-Z]',' ', review)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))  
    good_words = [w for w in words if not w in stops]
    return(' '.join(good_words))

In [30]:
# clean up train reviews

num_reviews = len(X_train)
clean_train_reviews = []
for b in range(0, num_reviews):
    clean_train_reviews.append(review_to_words(X_train[b]))

In [31]:
clean_train_reviews[3]

'poured transparent burnt orangish color nice fluffy white cap staying power head rentention pretty good big rich toasted pumpkin pumpkin pie come mind healthy dash fall spices leaning towards allspice nutmeg cinnamon nice blend toasted pumpkin pumpkin puree upfront initial booze sting fall spices light dash pepper bit earthy yeasiness underlying finish continues fresh toasted pumpkins nutmeg expect dry full little sweetness finish medium bodied nice little zing mixture spice booze sits palate well carbonated anf full effervescent overcarbonated like alot pumpkin ales one best pumpkin ales quite time huge fan style know good one taste brewers description pumpkin saison'

In [19]:
# vectorize the train data, fit and transform into feature vectors
vectorizer = CountVectorizer(analyzer='word')
train_data_features = vectorizer.fit_transform(clean_train_reviews)
# arrays are easy to work with, so:
train_data_features = train_data_features.toarray()



In [8]:
vocab = vectorizer.get_feature_names()
print(vocab)

['aa', 'aaa', 'aaaaaah', 'aaaaand', 'aaaack', 'aaaah', 'aaaahs', 'aaaand', 'aaah', 'aaahh', 'aaaiiyeeee', 'aachen', 'aachener', 'aack', 'aah', 'aaj', 'aal', 'aalborg', 'aals', 'aan', 'aand', 'aapa', 'aapect', 'aara', 'aardbei', 'aarde', 'aardwolf', 'aargh', 'aaroma', 'aaron', 'aaronf', 'aarrgh', 'aas', 'aasher', 'aass', 'aat', 'ab', 'aba', 'aback', 'abacus', 'abale', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandons', 'abarwithnoname', 'abas', 'abate', 'abated', 'abates', 'abating', 'abb', 'abbaye', 'abbe', 'abbey', 'abbeys', 'abbot', 'abbots', 'abbott', 'abbrasive', 'abbrassive', 'abbreviate', 'abbreviated', 'abbreviation', 'abbundantly', 'abby', 'abbyaye', 'abbys', 'abbywinters', 'abc', 'abcense', 'abd', 'abdicated', 'abdij', 'abdijbier', 'abdominal', 'abducting', 'abduction', 'abductions', 'abe', 'abeba', 'abel', 'abercrombie', 'aberdeen', 'aberlour', 'aberrartion', 'aberration', 'aberrations', 'abesentia', 'abet', 'abetted', 'abetting', 'abeyance', 'abf', 'abfod', 'abg

In [ ]:
dist = np.sum(train_data_features, axis=1)
for tag, count in zip(vocab, dist):
    print(count,tag)

341 aa
7 aaa
139 aaaaaah
0 aaaaand
59 aaaack
95 aaaah
30 aaaahs
0 aaaand
539 aaah
142 aaahh
242 aaaiiyeeee
0 aachen
0 aachener
0 aack
0 aah
89 aaj
36 aal
0 aalborg
201 aals
0 aan
90 aand
174 aapa
0 aapect
293 aara
233 aardbei
64 aarde
16 aardwolf
39 aargh
0 aaroma
722 aaron
382 aaronf
368 aarrgh
0 aas
78 aasher
178 aass
143 aat
0 ab
0 aba
761 aback
73 abacus
60 abale
8 abandon
30 abandoned
297 abandoning
240 abandonment
0 abandons
38 abarwithnoname
785 abas
0 abate
109 abated
266 abates
0 abating
184 abb
0 abbaye
44 abbe
75 abbey
252 abbeys
172 abbot
278 abbots
285 abbott
0 abbrasive
323 abbrassive
313 abbreviate
58 abbreviated
618 abbreviation
0 abbundantly
158 abby
20 abbyaye
0 abbys
0 abbywinters
40 abc
0 abcense
404 abd
0 abdicated
42 abdij
285 abdijbier
0 abdominal
45 abducting
0 abduction
0 abductions
213 abe
111 abeba
0 abel
213 abercrombie
0 aberdeen
0 aberlour
0 aberrartion
0 aberration
0 aberrations
0 abesentia
57 abet
475 abetted
83 abetting
0 abeyance
45 abf
46 abfod
0 abgb

0 agave
0 agawam
2 agd
251 age
57 ageable
71 aged
100 agedand
0 agedness
43 ageing
39 agency
0 agenda
0 agent
0 agentina
0 agents
65 agentzero
222 ages
0 aggessive
1692 aggies
98 aggravated
39 aggravating
0 aggregate
0 aggregated
88 aggregates
0 aggresive
0 aggresively
0 aggressice
0 aggression
49 aggressive
41 aggressively
26 aggressiveness
0 aggressivley
0 aggrevation
230 aggro
523 aghast
41 agian
141 agin
0 aging
73 agitate
624 agitated
131 agitates
523 agitating
0 agitation
0 agnes
367 agnostic
103 agnostics
146 ago
0 agonic
0 agonized
0 agonizing
59 agony
200 agood
0 agoras
0 agoura
0 agr
214 agradable
251 agradables
42 agragarian
315 agrainoftruthblog
579 agrarian
0 agree
755 agreeable
41 agreeableness
283 agreeably
56 agreed
0 agreeing
0 agreement
308 agrees
141 agreesive
0 agressive
16 agressively
198 agressivly
147 agrestic
0 agria
0 agricole
67 agricultural
56 agriculture
26 agrume
0 agrumes
0 agsin
101 agua
0 aguila
245 agur
447 ah
67 aha
52 ahaad
161 ahab
95 ahatcher
71 ahd

0 apa
63 apache
0 aparant
0 aparently
0 apariencia
84 aparris
0 apart
0 apartment
0 apas
382 apathetic
147 apathy
0 apatinska
0 apb
545 apbc
51 apbull
0 ape
424 apealing
59 apear
16 apearance
0 apears
1798 apect
141 apects
0 apeldoorn
0 apellidos
291 apels
4 apelsin
0 apendage
0 aperatif
0 aperitif
64 aperitiv
471 aperitivo
571 aperltif
492 apero
683 apertif
117 apesar
52 apeshit
338 apetizing
0 apex
144 apholstery
0 aphrodisiac
242 aphrodisiacs
0 aphrodisiaque
0 aphrodite
99 aphrorhum
0 apiana
46 apice
35 apiece
168 aping
161 apl
0 aplenty
0 aplle
36 aplomb
42 apls
73 apocalypse
0 apocalyptic
0 apocalypto
270 apocalyse
164 apocolypse
0 apogee
153 apollo
179 apologetic
66 apologetically
0 apologies
125 apologised
0 apologist
223 apologize
211 apologized
183 apologizing
275 apology
140 apolonia
203 apon
0 apophenia
0 apostol
85 apostrophe
522 apoth
16 apothecarian
79 apothecary
172 apotty
89 app
60 appalachia
59 appalachian
502 appalled
62 appalling
75 appamaddox
30 apparance
0 apparant

299 aschaffenburg
0 ascorbic
0 ascribe
0 ascribed
238 ascscents
72 asda
22 asds
633 asessionable
894 asf
250 asgreat
45 ash
123 ashai
217 ashame
0 ashamed
0 ashamedly
0 ashburn
55 ashburne
0 ashbury
51 asheley
0 ashen
171 asher
0 ashes
146 asheville
335 ashford
83 ashier
23 ashiness
1460 ashkelon
71 ashland
227 ashley
256 ashman
550 ashould
0 ashtray
71 ashtrays
127 ashy
1150 ashyness
0 asia
0 asiago
0 asian
0 asians
172 asid
0 aside
99 asii
227 asilky
204 asilomar
0 asinine
156 asint
0 asip
60 asit
169 ask
28 asked
87 askew
43 asking
0 asks
0 aslamm
23 aslan
58 asleep
0 aslight
63 aslin
290 aslo
15 asmall
0 asmf
0 asnd
226 aso
0 asoke
108 asolid
224 asome
108 asp
0 asparagus
67 aspartame
343 aspect
196 aspecto
22 aspects
0 aspen
0 aspergillus
45 asphalt
35 asphalty
0 aspirant
48 aspirating
116 aspirational
270 aspirations
0 aspire
38 aspired
325 aspires
107 aspirin
109 aspiring
77 asprin
77 aspx
397 asringency
0 ass
361 assail
0 assailed
0 assailing
316 assam
26 assassin
353 assassina

146 azure
314 azureoval
0 azusa
102 azy
0 azz
0 azzaca
0 azzurro
0 azzz
66 ba
133 baa
67 baaa
51 baaadd
0 baalnced
115 baba
71 babana
34 babble
56 babbling
0 babbys
33 babe
0 babenbergerstrasse
45 babes
21 babette
63 babied
76 babies
30 babs
166 babw
0 baby
78 babyback
94 babybel
0 babying
0 babylon
11 babysitter
0 babysitting
45 bac
33 bacalao
63 bacardi
406 bacause
87 bacbone
0 bacchanal
184 bacchus
251 bacground
62 bach
39 bachelor
60 bachteria
0 bacillus
282 bacjground
243 back
0 backaging
0 backbeat
24 backberry
420 backbite
0 backboard
170 backbone
101 backboned
45 backbones
0 backbons
0 backburner
31 backcountry
269 backdoor
0 backdraft
0 backdrop
0 backdrops
0 backed
463 backend
0 backer
21 backers
718 backflips
142 backgroiund
0 backgrond
48 backgroud
0 background
50 backgrounded
242 backgroundish
59 backgrounds
182 backgroung
0 backhalf
198 backhand
31 backhanded
43 backhoe
175 backig
312 backign
0 backing
437 backings
116 backlash
177 backlight
22 backlighted
39 backlighting

...

0 staunch
262 staunchest
0 staunchly
52 stavanger
0 stave
276 staves
209 staving
0 staw
272 stawberries
0 stawberry
0 stawn
354 stawski
0 stay
93 stayed
80 stayer
393 staying
0 stays
331 staywhatyouare
256 std
144 ste
50 stead
0 steadfast
105 steadfastly
54 steadfastness
0 steadies
0 steadily
0 steadman
41 steady
96 steak
78 steakhouse
306 steaks
0 steaky
0 steal
86 stealing
0 steallar
0 steals
0 stealth
374 stealthily
0 stealthy
36 steam
54 steambier
335 steamboat
36 steamed
236 steamer
434 steamin
87 steaming
0 steamlined
44 steamplant
0 steampunk
0 steamrail
74 steamroll
26 steamrolled
0 steamroller
0 steamrollered
0 steams
1509 steamwhistle
0 steamworks
261 steamy
440 steardy
0 stearine
149 steasy
0 steed
28 steeeeelers
51 steel
0 steelback
29 steele
94 steeled
0 steeler
92 steelers
0 steelhead
141 steelheads
0 steelie
318 steeliness
0 steelmantown
159 steely
0 steems
0 steen
0 steenberge
347 steenhuffel
234 steep
76 steeped
295 steeping
126 steeple
0 steeples
126 steeps
551 steer


131 strug
0 struggle
0 struggled
0 struggles
57 struggling
139 struise
413 struisse
0 strumke
0 strumming
250 strung
134 struse
89 strussel
0 strut
124 strutter
0 strutting
76 stst
257 sttaw
138 stticky
70 sttill
76 sttled
0 sttles
0 stu
75 stuart
133 stub
165 stubbe
0 stubbed
198 stubbie
74 stubbies
168 stubbified
144 stubbing
0 stubborn
508 stubborningly
0 stubbornly
0 stubbornness
78 stubby
0 stube
317 stuberl
204 stubob
3 stubs
0 stucco
187 stuck
0 stud
981 studd
148 studded
0 student
49 students
0 studied
86 studio
4 studios
56 studious
145 studiously
0 study
0 studying
212 stuff
0 stuffcajunpeoplelike
107 stuffed
49 stuffer
147 stuffers
416 stuffin
0 stuffing
189 stuffs
0 stuffy
617 stuffyness
0 stuggles
66 stule
0 stumble
93 stumbled
0 stumblefoot
0 stumbles
17 stumblin
0 stumbling
35 stump
49 stumped
0 stumpknocker
54 stumpled
121 stumps
54 stumptown
81 stumpy
820 stun
109 stung
24 stunk
449 stunned
96 stunner
158 stunning
0 stunningly
0 stuns
0 stunt
41 stunted
0 stupendous
0 

0 suprior
32 supris
119 suprise
0 suprised
149 supriseing
339 suprises
296 suprising
48 suprisingly
18 suprisng
172 suprize
26 suprized
0 supurb
0 sur
633 surburbia
60 surcharge
62 surdyk
112 sure
19 surefire
0 surely
21 surer
0 surette
0 surettes
0 surf
0 surface
0 surfaced
901 surfacefoams
30 surfaces
0 surfaceward
57 surfacing
214 surfactants
0 surfeit
83 surfer
234 surfers
0 surfin
0 surfing
150 surfrider
224 surge
144 surged
147 surgeon
0 surgery
103 surges
0 surgical
0 surgically
26 surging
0 surliness
243 surly
0 surlyfest
42 surmise
0 surmised
0 surmounted
38 surname
0 surpass
0 surpassed
99 surpasses
532 surpassing
0 surpassingly
80 surpirsed
0 surpise
0 surpised
0 surpises
20 surpising
356 surpisingly
199 surplus
363 surprenant
52 surpresa
250 surpress
0 surpressed
0 surpringly
0 surprisd
0 surprise
0 surprised
213 surprisely
0 surprises
0 surprisigly
56 surprising
0 surprisingly
60 surprisly
0 surprisngly
0 surprize
62 surprized
0 surprizing
486 surprizingly
783 surprsing
0 

0 tangelo
258 tangelos
42 tangent
0 tangents
27 tangerin
60 tangerine
229 tangerined
45 tangeriner
0 tangerines
0 tangeriney
694 tangering
0 tangey
54 tangibility
0 tangible
0 tangibles
44 tangier
0 tangiest
0 tanginess
418 tangle
87 tangled
100 tangles
268 tangletown
668 tangling
72 tango
393 tangoed
123 tangs
0 tangy
20 tangyness
0 tanhauser
0 tanhead
20 tanic
26 tanin
0 tanins
152 tanish
264 tank
0 tankard
153 tankards
0 tanked
0 tankenstein
352 tanker
2 tankhouse
555 tanking
0 tanks
0 tankstelle
1405 tanlines
0 tanned
0 tannenbaum
506 tannenbuam
0 tannens
0 tannenz
502 tanner
534 tannic
113 tannin
179 tanning
0 tannins
28 tanniny
340 tannis
0 tannish
98 tannnic
0 tanny
180 tanqueray
93 tans
412 tant
0 tantalise
178 tantalising
145 tantalite
0 tantalize
94 tantalizes
807 tantalizing
0 tantalizingly
24 tantilize
0 tantilizing
362 tantilizingly
46 tany
0 tanygy
132 tanzania
53 tanzanian
489 tanzelfestbier
0 tao
1129 taos
26 taosted
397 taosty
0 tap
0 tapa
0 tapas
0 tapat
0 tapboard
0 

188 thankfully
468 thanking
46 thankless
293 thanks
14 thanksgivign
231 thanksgiving
26 thankyou
207 thankyouverymuch
0 thant
112 thanx
242 thar
164 thas
249 thasn
0 thaste
191 thatch
458 thatcontinues
0 thate
0 thatlots
0 thatn
0 thats
0 thatsthatdude
31 thatt
36 thatwineguy
76 thaw
50 thawed
236 thay
97 thb
0 thbc
71 thcik
58 thck
210 thd
93 thea
0 theakson
155 theakston
0 theat
378 theater
153 theatre
94 theatric
421 theatrical
35 thebackbone
0 thebeer
0 thebeeralmanac
50 thebeerprophet
159 thebeers
0 thebeerspot
0 thebes
54 thebomber
588 thebouch
0 thebruery
347 thebugs
0 thecheeseman
0 thechrislife
10 thecitrus
90 theclassical
90 thecpj
54 thee
577 theee
0 theel
20 theepeeist
159 theese
0 theeveningbrews
0 thefatkid
0 theflavorbomb
183 theflavours
74 thefore
0 theft
331 theglass
0 thegr
18 thegrapefruit
0 theh
512 thehammer
0 thehand
0 thehops
128 thei
0 theie
63 theif
315 theil
0 theimpression
0 theior
10 theirselves
0 theist
0 thel
111 thelagerspiracy
0 thelongbeachbum
0 themani

In [32]:
text_clf= Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf',MultinomialNB()) ])

In [33]:
# first pass, .1195
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.11948877468217474

In [ ]:
###
# this takes too long, hours at least.
from sklearn.ensemble import RandomForestClassifier as RFC
forest = RFC(n_estimators=10)
forest.fit(train_data_features, y_train)

In [ ]:
num_reviews = len(X_test)
clean_test_reviews = []
for b in range(0, num_reviews):
    clean_test_reviews.append(review_to_words(X_test[b]))
    
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
result = forest.predict(test_data_features)
output = pd.DataFrame(data = {'id':test['id'], 'style':result})
output_to_csv('BagOfWords_model.csv', index=False, quoting=3)

In [116]:
df['review'][2]

'nice auburn impressions, tons of clarity, solid inch of off white head.   aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied.  nothing almond came out of it that was obvious.  kind of a fancied up oktoberfest that while good, didn\'t really change anything from the use of almonds or whatnot.   amber appearance, remarkable clarity. kinda that boring biscuity nothing happening nose to it. you\'d have to search to call it nutty. additionally, the taste was largely bland, minor residual sweetness, at no point could i discern anything \'almond\' about this beer. these \'hints of marzipan and almonds\' are at best hints, i found them to be nonexistant. really just came off like a sweet marzen/oktoberfest with nothing unique in the mix. seems like the almond addition was nice in theory, but it didn\'t show up on my palate. drank at the sierra nevada 30th anniversary party. had this at the tap room inside t

In [117]:
# vectorize a single review.
X = [df['review'][2]]  # start with a long review

vectorizer = CountVectorizer()
vectorizer.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [118]:
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [119]:
X2 = [df['review'][0]] 

vectorizer = CountVectorizer()
vectorizer.fit(X)
vector = vectorizer.transform(X2)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

(1, 384)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 0 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 2 0 0 0 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 2 0 0 0 2 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 0 2 7 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0 1 0 0 0 1 0 0 0 0 4 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [120]:
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [121]:
vector = vectorizer.transform(X2)
print(vector.toarray())

[[0 0 0 0 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 2 0 0 0 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 2 0 0 0 2 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 0 2 7 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0 1 0 0 0 1 0 0 0 0 4 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(X)
# summarize
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [113]:
print(vectorizer.idf_)

[ 6.20350706  7.90825515  7.90825515 ...,  7.50279005  7.90825515
  7.90825515]


In [98]:
# encode document
vector = vectorizer.transform([X[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 384)
[[ 0.01357063  0.01357063  0.0407119   0.01357063  0.02714126  0.01357063
   0.01357063  0.01357063  0.01357063  0.01357063  0.02714126  0.05428253
   0.13570632  0.01357063  0.01357063  0.01357063  0.08142379  0.01357063
   0.01357063  0.01357063  0.02714126  0.36640707  0.01357063  0.01357063
   0.02714126  0.01357063  0.01357063  0.0407119   0.01357063  0.01357063
   0.14927696  0.01357063  0.01357063  0.01357063  0.02714126  0.01357063
   0.01357063  0.01357063  0.01357063  0.05428253  0.13570632  0.01357063
   0.01357063  0.01357063  0.02714126  0.01357063  0.01357063  0.01357063
   0.01357063  0.01357063  0.02714126  0.0407119   0.01357063  0.01357063
   0.01357063  0.0407119   0.02714126  0.0407119   0.01357063  0.10856506
   0.02714126  0.02714126  0.02714126  0.01357063  0.01357063  0.01357063
   0.0407119   0.02714126  0.01357063  0.01357063  0.01357063  0.01357063
   0.02714126  0.06785316  0.0407119   0.01357063  0.0407119   0.02714126
   0.0407119   0.01357063  0.

In [104]:
# vectorize with a list of reviews

from sklearn.feature_extraction.text import TfidfVectorizer
X = df['review'][0:2000]

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(X)
# summarize
print(str(vectorizer.vocabulary_)[:1000])  # print a slice because this is very long
print(len(str(vectorizer.vocabulary_)))

{'smell': 11165, 'soft': 11263, 'hop': 6090, 'aroma': 918, 'with': 13621, 'significant': 10951, 'malt': 7438, 'scents': 10579, 'this': 12373, 'one': 8499, 'smells': 11171, 'very': 13165, 'creamy': 3179, 'taste': 12183, 'and': 760, 'it': 6575, 'is': 6562, 'the': 12317, 'traditional': 12594, 'irish': 6546, 'flavors': 4964, 'come': 2790, 'out': 8606, 'at': 1005, 'tongue': 12522, 'not': 8299, 'like': 7146, 'cream': 3171, 'ale': 629, 'but': 2030, 'close': 2653, 'big': 1452, 'buttery': 2039, 'smooth': 11194, 'hops': 6116, 'are': 896, 'unique': 12939, 'sharp': 10812, 'flavor': 4957, 'an': 750, 'easy': 4133, 'saturated': 10524, 'well': 13435, 'mixed': 7865, 'blend': 1560, 'that': 12312, 'plays': 9177, 'complimenting': 2866, 'second': 10666, 'fiddle': 4818, 'to': 12482, 'base': 1222, 'no': 8248, 'sweetness': 12034, 'finish': 4878, 'nutty': 8357, 'changes': 2388, 'personalities': 9001, 'end': 4289, 'mouthfeel': 8004, 'lightly': 7135, 'carbonated': 2203, 'exceptionally': 4515, 'drinkability': 400

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer

X = [df['review'][2]]  # start with a long review
X2 = [df['review'][0]] # X2 is shorter

# create the transform
vectorizer = HashingVectorizer(n_features=50)
# encode document
vector = vectorizer.transform(X)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 50)
[[ 0.07887912  0.01314652 -0.0657326   0.02629304 -0.0657326  -0.17090475
  -0.01314652  0.         -0.28922343  0.01314652  0.10517215  0.01314652
   0.28922343 -0.14461171 -0.01314652  0.01314652 -0.02629304 -0.10517215
   0.21034431  0.          0.07887912  0.14461171 -0.07887912  0.02629304
  -0.21034431  0.07887912 -0.14461171  0.14461171  0.01314652  0.
   0.02629304 -0.03943956  0.24978387  0.14461171  0.07887912  0.15775823
   0.07887912 -0.02629304  0.23663735  0.13146519 -0.22349083  0.19719779
   0.         -0.03943956  0.07887912 -0.36810254  0.36810254 -0.07887912
  -0.03943956  0.01314652]]


In [31]:
counts = df.groupby(['style']).size()
styles = counts.index

In [32]:
styles

Index(['Altbier', 'American Adjunct Lager', 'American Amber / Red Ale',
       'American Amber / Red Lager', 'American Barleywine',
       'American Black Ale', 'American Blonde Ale', 'American Brown Ale',
       'American Dark Wheat Ale', 'American Double / Imperial IPA',
       ...
       'Scotch Ale / Wee Heavy', 'Scottish Ale',
       'Scottish Gruit / Ancient Herbed Ale', 'Smoked Beer', 'Tripel',
       'Vienna Lager', 'Weizenbock', 'Wheatwine', 'Winter Warmer', 'Witbier'],
      dtype='object', name='style', length=104)

In [51]:
# vectorize with a list of reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#  Create X for predictor variables, y for target
X = df['review'][0:2000].values
y = df['style'][0:2000].values
test_X = df['review'][3000:4000].values
test_y = df['style'][3000:4000].values

# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X)

# Naive Bayes classifier
clf = MultinomialNB().fit(X_train, y)

In [52]:
predicted = clf.predict(test_X)
np.mean(predicted == test_y)

ValueError: could not convert string to float: "   poured from a 650ml bottle,7% abv, 80 ibu's fresh hopped with cascade and centennial with nugget for bittering. l: pours a dark red amber with just a skim of head with minor lacing. s: not much of a nose for me ,some hops and mild spice. t: really smooth well balance red ipa, looks like they used 5 malts and it works well with the fresh hops nice on the bitterness. f: a light carbonation nice mouth feel no alcohol taste at all. o: a really nice fresh hopped red ipa.   "

In [ ]:
# could use a pipeline to do the same:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),                      
                     ('clf', MultinomialNB()),])
text_clf = text_clf.fit(X,y)

### start over here

In [60]:
df = df_copy

In [66]:
#prep the data
df = df_copy
df = df.drop(['name','brewery'], axis=1)
# drop all reviews with < 15 characters
df = df[df['review'].map(len) > 14]

In [68]:
print(len(df))

49291


In [69]:
t_id = 0
text = df.loc[t_id, 'review']

In [72]:
from gensim.summarization import summarize, keywords

word_scores = keywords(text, words=5, scores=True, split=True, lemmatize=True)

ModuleNotFoundError: No module named 'gensim'